In [1]:
## Constructs script for allignment to H99 Reference genome.
## uses SAMTOOLS and FREEBAYES
## Collect the progeny fastq names
## Set local directories and pahts.
## These include local dir and file names on Magwene lab machines
## and will need to augmented for use elsewhere
## Construct variables using data on local machine

## Import needed modules. 
import os, pandas as pd, numpy as np
#from os import listdir
#from os.path import isfile, join

In [2]:
## Gather current working directories
scriptsdir = os.getcwd()

In [3]:
## Load in reference file and parese out contig names
## Set path to reference
ref_file_path = '../DATA/H99_latest_Vikas_assembly.fasta'

## open file parse lines
pb = [line for line in open(ref_file_path,'r').readlines()]

## Check work
assert len(pb) > 0, "Missing reference file and contents."

## Gather contig names
contig_names = [l[1:-1] for l in pb if l[0] == '>']

## Check length and print 5 contig names.
assert len(contig_names) >= 14, "Missing a contig!"
contig_names[:5]

['Chr14', 'Chr13', 'Chr12', 'Chr11', 'Chr10']

In [4]:
## Load in list of fast q files used in analysis
bts = [l.split('\n')[0] for l in open('../DATA/Progeny_FASTQS.csv','r').readlines()]

## Print length and names of files
len(bts), bts[-4:]

(56,
 ['Bt65F1prog44_S63_L006_R1_001.fastq.gz',
  'Bt65F1prog44_S63_L006_R2_001.fastq.gz',
  'Bt65F1prog8_S54_L006_R1_001.fastq.gz',
  'Bt65F1prog8_S54_L006_R2_001.fastq.gz'])

In [5]:
## Gather progeny integer names
prog_int = np.unique([int(b.split('prog')[-1].split('_S')[0]) for b in bts])

## Print a few four and len
prog_int[-4:], len(prog_int)

(array([37, 39, 41, 44]), 28)

In [6]:
## Gather progeny names
progs = sorted(np.unique([b.split('R')[0] for b in bts]))

## Check work
assert len(progs) == len(prog_int)

In [7]:
## Gather file ends of fastqs
file_ends = np.unique(['R'+b.split('R')[-1] for b in bts])

## Check work
assert len(file_ends) == 2, "Error, fastq files are not paired, check files."

## Print file end
file_ends

array(['R1_001.fastq.gz', 'R2_001.fastq.gz'], dtype='<U15')

In [8]:
## Write variables 
## These vairalbes will include paths to files and executables on Haraka
## File names
my_h99_align = '/Crypto-HM-H99-aln-pe-sam.sh' #1st in pipeline
my_sam_to_bam = '/Crypto-HM-H99-Bio-sam-to-sorted-bam.sh' # 2nd in pipeline
my_bamaddrg = '/Crypto-HM-H99-Bio-bamaddrg.sh' # 3rd in pipeline
myfreebay = ['/Crypto-HM-H99-Bio-freebayes1.sh',
             '/Crypto-HM-H99-Bio-freebayes2.sh',
             '/Crypto-HM-H99-Bio-freebayes3.sh'] # 4th in pipeline

## List of sorted bam files with read group info. 
mybamsrg = '/Crypto-HM-H99-list-sort-bams-rg.txt'

## Script directory 
duck_SCRIPTS = '/bigscratch0/croth/HYPERMUTATOR/Hypermutator/SCRIPTS'
duck_VCF = '/bigscratch0/croth/HYPERMUTATOR/Hypermutator/VCFs/H99'

## duckydog paths
duck_SHEBANG = '#!/bin/bash\n'

## Reference
duck_REF = '/bigscratch0/croth/HYPERMUTATOR/Hypermutator/REFs/%s'%(ref_file_path.split('/')[-1])

## QTL raw fastq file directory 
duck_QTL_RAW = '/bigscratch0/croth/HYPERMUTATOR/Hypermutator/Priest_4915_180622A1'

## SAMS and BAMS directories 
duck_SAMS = '/bigscratch0/croth/HYPERMUTATOR/Hypermutator/SAM/H99/'
duck_BAMS = '/bigscratch0/croth/HYPERMUTATOR/Hypermutator/BAM/H99/'

## commands
duck_BWA = 'bwa mem -v 0 ' + duck_REF

## Upload these files to and make executable via chmod, e.g. "chmod +x Crypto-DNX-Pac_Bio_align.sh"

In [9]:
## Align
## Open and write script for aligning reads
f = open(scriptsdir+my_h99_align,'w') ## open file with samples to be remapped (b/c of their corrected parttners)
print(duck_SHEBANG,file=f) ## print the shebang
sams = []
for seg in progs:
    print(duck_BWA, ## print the bwa command and reference genome
          duck_QTL_RAW+ '/'+seg+file_ends[0], ## the first read in pair file 
          duck_QTL_RAW + '/'+seg+file_ends[1],'>', ## the second
          duck_SAMS+seg+duck_REF.split('/')[-1].split('.')[0]+'-aln-pe.sam\n', ## The final sam file
        file=f) ## tells print which file to print to. Que clever ;)
    sams.append(duck_SAMS+seg+duck_REF.split('/')[-1].split('.')[0]+'-aln-pe.sam')
f.close() ## close the file

In [10]:
## Sam to Bam
## check work
assert len(sams) == len(prog_int), "Missing sam files!"

## Show five file paths
sams[:5]

['/bigscratch0/croth/HYPERMUTATOR/Hypermutator/SAM/H99/Bt65F1prog11_S64_L006_H99_latest_Vikas_assembly-aln-pe.sam',
 '/bigscratch0/croth/HYPERMUTATOR/Hypermutator/SAM/H99/Bt65F1prog12_S39_L005_H99_latest_Vikas_assembly-aln-pe.sam',
 '/bigscratch0/croth/HYPERMUTATOR/Hypermutator/SAM/H99/Bt65F1prog13_S65_L006_H99_latest_Vikas_assembly-aln-pe.sam',
 '/bigscratch0/croth/HYPERMUTATOR/Hypermutator/SAM/H99/Bt65F1prog14_S66_L006_H99_latest_Vikas_assembly-aln-pe.sam',
 '/bigscratch0/croth/HYPERMUTATOR/Hypermutator/SAM/H99/Bt65F1prog17_S67_L006_H99_latest_Vikas_assembly-aln-pe.sam']

In [11]:
## SAM to BAM conversion
f = open(scriptsdir+my_sam_to_bam,'w') ## Open file to print text to ... 
bams = []
print(duck_SHEBANG,file=f) ## print the shebang
for sam in sams:
    bam = duck_BAMS+sam.split('/')[-1].split('.')[0]+'-sorted'
    print('samtools view -bS %s | samtools sort - %s'%(sam,bam+'\n'),file=f) ## Make sam to bam file
    bams.append(bam+'.bam')
f.close()

In [12]:
## Bam to Bamaddrg
bams[:5]

['/bigscratch0/croth/HYPERMUTATOR/Hypermutator/BAM/H99/Bt65F1prog11_S64_L006_H99_latest_Vikas_assembly-aln-pe-sorted.bam',
 '/bigscratch0/croth/HYPERMUTATOR/Hypermutator/BAM/H99/Bt65F1prog12_S39_L005_H99_latest_Vikas_assembly-aln-pe-sorted.bam',
 '/bigscratch0/croth/HYPERMUTATOR/Hypermutator/BAM/H99/Bt65F1prog13_S65_L006_H99_latest_Vikas_assembly-aln-pe-sorted.bam',
 '/bigscratch0/croth/HYPERMUTATOR/Hypermutator/BAM/H99/Bt65F1prog14_S66_L006_H99_latest_Vikas_assembly-aln-pe-sorted.bam',
 '/bigscratch0/croth/HYPERMUTATOR/Hypermutator/BAM/H99/Bt65F1prog17_S67_L006_H99_latest_Vikas_assembly-aln-pe-sorted.bam']

In [13]:
f = open(scriptsdir+my_bamaddrg,'w') ## Open file to print text to ... 
print(duck_SHEBANG,file=f) ## print the shebang
print('cd /bigscratch0/croth/bamaddrg\n',file=f)
bamsrg = []
for bam in bams:
    print('./bamaddrg -b %s -s %s -r %s > %s\n'%(
        bam,
        bam.split('/Bt65F1')[-1].split('_L')[0],
        bam.split('/')[-1].split('_H99')[0],
        bam.split('.')[0]+'-rg.bam'
    ),file=f) 
    print('rm %s\n'%(bam),file=f)
    print('samtools index %s %s'%(bam.split('.')[0]+'-rg.bam',## Use samtools to index bam file
                                  bam.split('.')[0]+'-rg.bam.bai\n'),file=f) 
    print('echo added rg to %s'%(bam.split('/')[-1].split('_H99')[0]),file=f)
    bamsrg.append(bam.split('.')[0]+'-rg.bam')
f.close()

In [14]:
## FREEBAYES!!!!!
## Construct commands for freebayes variant caller
## Check work, 
assert len(bamsrg) == len(progs), "Missing sorted BAM files"

## Print bams to file
f = open(scriptsdir+mybamsrg,'w')
for bam in bamsrg:
    print(bam,file=f)
f.close()

In [15]:
## Split analysis of variants across chromosomes
## Patch list of chromosomes
chrom_map = [a for a in contig_names]

## SPlit chromosomes across freebayes bash commands
chrom_file_maps = [chrom_map[i::len(myfreebay)] for i in range(len(myfreebay))]

## CHeck work
assert len(np.unique(np.concatenate(chrom_file_maps))) == 14, "Error in chromosome sorting"

In [16]:
## Make and write feebayes commands
## Iterate over freebaye bash scripts
for i,path in enumerate(myfreebay):
    
    ## Gather chromosomes for analysis
    chrom_map = chrom_file_maps[i]
    
    ## Open file
    f = open(scriptsdir+path,'w')
    
    ## Iterate over chromosomes
    for region in chrom_map:
        
        ## Write the freebayes command for this chromosome
        print(region)
        new_vcf = duck_VCF+'/Bt65F1prog-'+region+'-'+duck_REF.split('.fasta'
                )[0].split('/')[-1]+'-'+str(len(bamsrg))+'.vcf'
        freebayes = '/usr/local/bin/freebayes -f %s -p %s -r %s -L %s -= > %s'%(
        duck_REF,str(1),region,duck_SCRIPTS+mybamsrg,new_vcf)
        
        print(freebayes,file=f)
        print('gzip %s'%new_vcf,file=f)
        print(' ',file=f)
        
    f.close()

Chr14
Chr11
Chr8
Chr5
Chr2
Chr13
Chr10
Chr7
Chr4
Chr1
Chr12
Chr9
Chr6
Chr3
